## Importamos las dependencias y librerias

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

## Preparamos los datos y los parametros

In [4]:
data_dir = "/content/drive/MyDrive/datasets/pizza_not_pizza"

# Parámetros básicos
batch_size = 32
img_size = 180  # Ancho y alto de las imágenes
num_epochs = 10
validation_split = 0.2
random_seed = 123


In [5]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Valores típicos para imágenes RGB
                         std=[0.229, 0.224, 0.225])
])

# Cargar el dataset completo usando ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# División del dataset en entrenamiento y validación
total_size = len(dataset)
val_size = int(total_size * validation_split)
train_size = total_size - val_size

torch.manual_seed(random_seed)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Crear DataLoaders para entrenamiento y validación
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Modelo

In [6]:
# Definir un modelo CNN simple
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # Reducción a la mitad
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # Reducción a la cuarta parte
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)   # Reducción a la octava parte
        )
        # Calcular el tamaño de la característica tras las capas convolucionales
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * (img_size // 8) * (img_size // 8), 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Configurar dispositivo: GPU si está disponible, de lo contrario CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=2).to(device)

# Función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Función de entrenamiento

In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        # Entrenamiento
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size

        # Evaluación en el conjunto de validación
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / val_size
        val_acc = val_corrects.double() / val_size

        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} - "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


In [8]:
# Iniciar el entrenamiento del modelo
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch 1/10 - Train Loss: 0.6436, Train Acc: 0.6529 - Val Loss: 0.5496, Val Acc: 0.7379
Epoch 2/10 - Train Loss: 0.5826, Train Acc: 0.7025 - Val Loss: 0.5640, Val Acc: 0.7023
Epoch 3/10 - Train Loss: 0.5026, Train Acc: 0.7661 - Val Loss: 0.5197, Val Acc: 0.7481
Epoch 4/10 - Train Loss: 0.4619, Train Acc: 0.7921 - Val Loss: 0.4973, Val Acc: 0.7583
Epoch 5/10 - Train Loss: 0.3393, Train Acc: 0.8576 - Val Loss: 0.4851, Val Acc: 0.7837
Epoch 6/10 - Train Loss: 0.2255, Train Acc: 0.8989 - Val Loss: 0.5512, Val Acc: 0.7837
Epoch 7/10 - Train Loss: 0.1183, Train Acc: 0.9593 - Val Loss: 0.6481, Val Acc: 0.7761
Epoch 8/10 - Train Loss: 0.0606, Train Acc: 0.9797 - Val Loss: 0.8281, Val Acc: 0.7837
Epoch 9/10 - Train Loss: 0.0290, Train Acc: 0.9930 - Val Loss: 0.8308, Val Acc: 0.7710
Epoch 10/10 - Train Loss: 0.0240, Train Acc: 0.9917 - Val Loss: 1.4826, Val Acc: 0.7328


## Evaluación

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

def test_model(model, val_dataset, num_images=25):
    model.eval()
    fig = plt.figure(figsize=(10, 8))

    # Obtener índices aleatorios de imágenes
    random_indices = random.sample(range(len(val_dataset)), num_images)

    with torch.no_grad():
        for i, index in enumerate(random_indices):
            # Obtener la imagen y la etiqueta usando el índice aleatorio
            image, label = val_dataset[index]

            # Preparar la imagen y moverla al dispositivo correspondiente
            image = image.unsqueeze(0).to(device)
            label = torch.tensor([label]).to(device)

            outputs = model(image)
            _, preds = torch.max(outputs, 1)

            ax = plt.subplot(5, 5, i + 1)  # Organiza las imágenes en una cuadrícula 5x5
            ax.axis('off')
            ax.set_title(f'Predicted: {dataset.classes[preds[0]]}\nActual: {dataset.classes[label.item()]}', fontsize=8)

            # Mover la imagen a CPU para mostrarla
            image = image.cpu().squeeze().numpy().transpose((1, 2, 0))
            # Desnormalizar para mostrar correctamente
            image = (image * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
            image = np.clip(image, 0, 1)

            ax.imshow(image)

    # Ajustar el espacio entre subplots para evitar solapamientos
    plt.subplots_adjust(wspace=0.5, hspace=0.5)

# Uso del val_dataset como test_loader (idealmente se debería tener un conjunto de test separado)
test_model(model, val_dataset)
plt.show()


NameError: name 'model' is not defined